# Investigation: Sweden "Internal" Transfers 🇸🇪⚽

Same check as Liga MX but for Sweden. **Spoiler:** same result — zero internal transfers exist, all records are inter-league (Allsvenskan ↔ Superettan).

In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 200)

BASE_PATH = "../../thesis_data/"
df_comp = pd.read_parquet(f"{BASE_PATH}raw_data_twelve/Wyscout/competitions_wyscout.parquet")
df = pd.read_parquet(f"{BASE_PATH}raw_data_twelve/Twelve/male_transfer_model.parquet")
comp_id_to_name = df_comp.drop_duplicates('competition_id').set_index('competition_id')['name'].to_dict()

## 1) Swedish Competitions in Dataset

In [2]:
ALLSVENSKAN = 808
SUPERETTAN = 818
SWE_IDS = [ALLSVENSKAN, SUPERETTAN]

swe_comps = df_comp[df_comp['country'] == 'Sweden']
swe_comps.groupby(['competition_id', 'name']).agg(
    n_seasons=('season', 'nunique'),
    seasons=('season', lambda x: sorted(x.unique()))
).reset_index()

,competition_id,name,n_seasons,seasons
0,808,Allsvenskan,8,"[2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]"
1,818,Superettan,8,"[2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]"


## 2) Zero Internal Transfers

In [3]:
# Global check
same_comp = (df['from_competition'] == df['to_competition']).sum()
print(f"Records where from_competition == to_competition (entire dataset): {same_comp}")
print(f"\n\u274c Zero internal transfers for ANY league, including Allsvenskan.")

Records where from_competition == to_competition (entire dataset): 0

❌ Zero internal transfers for ANY league, including Allsvenskan.


## 3) What ARE the Sweden → Sweden Transfers?

In [4]:
swe = df[
    (df['from_competition'].isin(SWE_IDS)) & (df['to_competition'].isin(SWE_IDS))
].copy()

swe['from_name'] = swe['from_competition'].map(comp_id_to_name)
swe['to_name'] = swe['to_competition'].map(comp_id_to_name)
swe['same_team'] = swe['from_team_id'] == swe['to_team_id']

print(f"Total Sweden\u2192Sweden records: {len(swe)}")
print(f"  Same team (promotion/relegation): {swe['same_team'].sum()}")
print(f"  Different team (player moved clubs between divisions): {(~swe['same_team']).sum()}")

# Breakdown
swe.groupby(['from_competition', 'to_competition', 'from_name', 'to_name']).agg(
    n_records=('player_id', 'count'),
    n_same_team=('same_team', 'sum')
).reset_index()

Total Sweden→Sweden records: 486
  Same team (promotion/relegation): 325
  Different team (player moved clubs between divisions): 161


,from_competition,to_competition,from_name,to_name,n_records,n_same_team
0,808,818,Allsvenskan,Superettan,189,134
1,818,808,Superettan,Allsvenskan,297,191


In [5]:
# competition_id is stable across seasons
from_808 = df[df['from_competition'] == ALLSVENSKAN]
to_808 = df[df['to_competition'] == ALLSVENSKAN]

print(f"Allsvenskan (808) FROM seasons: {sorted(from_808['from_season'].unique())}")
print(f"Allsvenskan (808) TO seasons:   {sorted(to_808['to_season'].unique())}")
print(f"\n\u2705 Same ID across all seasons. competition_id = league, not season.")

Allsvenskan (808) FROM seasons: [np.int16(2018), np.int16(2019), np.int16(2020), np.int16(2021), np.int16(2022), np.int16(2023), np.int16(2024), np.int16(2025)]
Allsvenskan (808) TO seasons:   [np.int16(2018), np.int16(2019), np.int16(2020), np.int16(2021), np.int16(2022), np.int16(2023), np.int16(2024), np.int16(2025)]

✅ Same ID across all seasons. competition_id = league, not season.


---
## 🎯 Conclusion

| Fact | Value |
|------|-------|
| Internal Allsvenskan transfers (808→808) | **0** |
| Sweden→Sweden records | 486 |
| Of which: same team (promo/releg) | 325 |
| Of which: diff team (cross-division) | 161 |
| competition_id stable across seasons? | **Yes** |

All 486 Sweden→Sweden records are **Allsvenskan ↔ Superettan** (1st ↔ 2nd division). None are internal transfers within the same league. The dataset by design only contains inter-league transfers.